# Recommender using Embeddings

Data taken from https://www.kaggle.com/datasets/mexwell/us-software-engineer-jobs

In [1]:
import pandas as pd
from pathlib import Path
file = 'us-software-engineer-jobs-zenrows.csv'

df = pd.read_csv(Path('data',file))
df.shape

(58433, 29)

In [2]:
df.describe()

,rating,review_count,ad_id,source_id,job_location_postal
count,58433.000000,58433.000000,2.362600e+04,5.843300e+04,26121.000000
mean,2.565040,1604.469050,3.376991e+08,6.605754e+06,52579.873627
std,1.832927,5894.934474,8.307966e+07,7.333885e+06,33371.519030
min,0.000000,0.000000,9.826900e+04,1.700000e+01,603.000000
25%,0.000000,0.000000,3.616971e+08,1.571000e+04,20877.000000
50%,3.500000,14.000000,3.694296e+08,3.370807e+06,53122.000000
75%,4.000000,624.000000,3.716887e+08,1.307450e+07,87124.000000
max,5.000000,223345.000000,3.725696e+08,2.077308e+07,99901.000000


In [3]:
columns = ['title', 'company', 'types', 'location', 'snippet']
df = df[columns]
df.shape

(58433, 5)

In [4]:
df[df['snippet'].isnull()].dropna(inplace=True, axis='rows')

/var/folders/7m/74_ct3hx33d878n626w1wxyc0000gn/T/ipykernel_32415/96582268.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['snippet'].isnull()].dropna(inplace=True, axis='rows')


In [5]:
# Filter out rows 
mask = df[['title', 'snippet']].isnull().any(axis='columns')
df[mask].index

Index([928, 2796, 24639, 36774, 38167, 39357, 44512, 57079], dtype='int64')

In [6]:
df.drop(index=df[mask].index, inplace=True)
df.shape

(58425, 5)

In [7]:
df.sample(5)

,title,company,types,location,snippet
40628,Software Engineer - Crypto Products,TradeStation,NaN,Remote,Work and collaborate with DevOps engineers to ...
56524,Principal Software Engineer,Liberty Mutual Insurance,NaN,Remote,7+ years of software engineering experience.\n...
32993,C# Senior Software Developer (Contractor) - Fr...,DataAxxis,"Full-time, Contract","New York, NY",The Front Office Senior Software Developer wil...
3546,Site Reliability Engineer,SHIELD AI,NaN,"San Diego, CA",Collaborate with a diverse group of supportive...
37609,c# Engineer,Optimum Technologies,Full-time,"San Francisco Bay Area, CA",Versed in software engineering best practices ...


In [22]:
pd.set_option('display.max_colwidth', None)

documents = list(df['snippet'][:5])

In [10]:
from fastembed.embedding import FlagEmbedding as Embedding
import numpy as np

Embedding.list_supported_models()

[{'model': 'BAAI/bge-small-en',
  'dim': 384,
  'description': 'Fast English model',
  'size_in_GB': 0.2},
 {'model': 'BAAI/bge-small-en-v1.5',
  'dim': 384,
  'description': 'Fast and Default English model',
  'size_in_GB': 0.13},
 {'model': 'BAAI/bge-small-zh-v1.5',
  'dim': 512,
  'description': 'Fast and recommended Chinese model',
  'size_in_GB': 0.1},
 {'model': 'BAAI/bge-base-en',
  'dim': 768,
  'description': 'Base English model',
  'size_in_GB': 0.5},
 {'model': 'BAAI/bge-base-en-v1.5',
  'dim': 768,
  'description': 'Base English model, v1.5',
  'size_in_GB': 0.44},
 {'model': 'sentence-transformers/all-MiniLM-L6-v2',
  'dim': 384,
  'description': 'Sentence Transformer model, MiniLM-L6-v2',
  'size_in_GB': 0.09},
 {'model': 'intfloat/multilingual-e5-large',
  'dim': 1024,
  'description': 'Multilingual model, e5-large. Recommend using this model for non-English languages',
  'size_in_GB': 2.24}]

In [23]:
embedding_model = Embedding(model_name="BAAI/bge-small-en-v1.5", max_length=512) 
embeddings: [np.ndarray] = list(embedding_model.embed(documents)) # Note the list() call - this is a generator 

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76.7M/76.7M [00:06<00:00, 11.6MiB/s]


In [35]:
def print_top_k(query, embeddings, documents, top_k=5):
    query_embedding = list(embedding_model.query_embed(query))[0]
    
    # Calculate cosine similarity.
    scores = np.dot(embeddings, query_embedding)
    
    sorted_scores = np.argsort(scores)[::-1]
    for i in range(top_k):
        print(f"Rank {i+1}: {documents[sorted_scores[i]]}")

In [36]:
print_top_k("I am a backend engineer looking for golang job", embeddings, documents)

Rank 1: This person will be a senior member of the team and will be responsible for architecting, building complex features and providing technical guidance to other…
Rank 2: Proficiency in Agile software development principles is required.
 Advanced knowledge of industry software development methodologies, standards and architecture…
Rank 3: The ideal candidate will have a skill for tough puzzles, a talent for communicating complex ideas simply, and a drive to meet high expectations with great…
Rank 4: Reports to* DIRECTOR OF MARKETING.
 PHP - Equivalent of 3 years education and/or experience.
 Java script - Equivalent of 3 years education and/or experience.
Rank 5: Throughout the day, you will collaborate with your teammates and interact with our clients.
 Benefits! Shockoe offers a comprehensive and competitive benefits…


In [37]:
print_top_k("I am a backend engineer looking for php job", embeddings, documents)

Rank 1: Reports to* DIRECTOR OF MARKETING.
 PHP - Equivalent of 3 years education and/or experience.
 Java script - Equivalent of 3 years education and/or experience.
Rank 2: This person will be a senior member of the team and will be responsible for architecting, building complex features and providing technical guidance to other…
Rank 3: The ideal candidate will have a skill for tough puzzles, a talent for communicating complex ideas simply, and a drive to meet high expectations with great…
Rank 4: Throughout the day, you will collaborate with your teammates and interact with our clients.
 Benefits! Shockoe offers a comprehensive and competitive benefits…
Rank 5: Proficiency in Agile software development principles is required.
 Advanced knowledge of industry software development methodologies, standards and architecture…
